In [2]:
import pandas as pd
import numpy as np
from scipy.io import arff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Function to load and preprocess data
def load_preprocess_data(file_path):
    data, meta = arff.loadarff(file_path)
    df = pd.DataFrame(data)
    
    # Convert byte string to normal string for object-type columns
    str_df = df.select_dtypes([np.object])
    str_df = str_df.stack().str.decode('utf-8').unstack()
    
    for col in str_df:
        df[col] = str_df[col]
    
    # Handle missing values
    df.fillna(df.median(), inplace=True)
    
    # Convert all columns to numeric
    df = df.apply(pd.to_numeric)

    return df

# Load and preprocess each file
file_paths = ['1year.arff', '2year.arff', '3year.arff', '4year.arff', '5year.arff']
all_data = pd.concat([load_preprocess_data(f'{file}') for file in file_paths])

# Display first few rows of the dataframe
print(all_data.head())


      Attr1    Attr2    Attr3   Attr4    Attr5    Attr6     Attr7    Attr8  \
0  0.200550  0.37951  0.39641  2.0472  32.3510  0.38825  0.249760  1.33050   
1  0.209120  0.49988  0.47225  1.9447  14.7860  0.00000  0.258340  0.99601   
2  0.248660  0.69592  0.26713  1.5548  -1.1523  0.00000  0.309060  0.43695   
3  0.081483  0.30734  0.45879  2.4928  51.9520  0.14988  0.092704  1.86610   
4  0.187320  0.61323  0.22960  1.4063  -7.3128  0.18732  0.187320  0.63070   

    Attr9   Attr10  ...    Attr56   Attr57   Attr58    Attr59  Attr60  Attr61  \
0  1.1389  0.50494  ...  0.121960  0.39718  0.87804  0.001924  8.4160  5.1372   
1  1.6996  0.49788  ...  0.121300  0.42002  0.85300  0.000000  4.1486  3.2732   
2  1.3090  0.30408  ...  0.241140  0.81774  0.76599  0.694840  4.9909  3.9510   
3  1.0571  0.57353  ...  0.054015  0.14207  0.94598  0.000000  4.5746  3.6147   
4  1.1559  0.38677  ...  0.134850  0.48431  0.86515  0.124440  6.3985  4.3158   

    Attr62  Attr63   Attr64  class  
0   82.

In [3]:
# Assuming the last column is the target variable
X = all_data.iloc[:, :-1]  # Features
y = all_data.iloc[:, -1]   # Target

# Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [4]:
# Building a Neural Network for Bankruptcy Prediction
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 6,273
Trainable params: 6,273
Non-trainable params: 0
_________________________________________________________________


2023-11-30 04:16:33.655197: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2023-11-30 04:16:33.676010: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3691525000 Hz
2023-11-30 04:16:33.676578: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1af5580 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-11-30 04:16:33.676601: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-11-30 04:16:33.676725: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.3f}, Test Accuracy: {accuracy:.3f}")

# Plotting training history
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss over Epochs')
plt.legend()
plt.show()


Train on 27779 samples, validate on 6945 samples
Epoch 1/100
27779/27779 [==============================] - 5s 189us/sample - loss: 0.2352 - accuracy: 0.9523 - val_loss: 0.1945 - val_accuracy: 0.9497
Epoch 2/100
27779/27779 [==============================] - 4s 144us/sample - loss: 0.1940 - accuracy: 0.9530 - val_loss: 0.1884 - val_accuracy: 0.9499
Epoch 3/100
27779/27779 [==============================] - 4s 151us/sample - loss: 0.1889 - accuracy: 0.9530 - val_loss: 0.1880 - val_accuracy: 0.9497
Epoch 4/100
27779/27779 [==============================] - 5s 162us/sample - loss: 0.1860 - accuracy: 0.9530 - val_loss: 0.1878 - val_accuracy: 0.9496
Epoch 5/100
27779/27779 [==============================] - 4s 139us/sample - loss: 0.1807 - accuracy: 0.9532 - val_loss: 0.1853 - val_accuracy: 0.9496
Epoch 6/100
27779/27779 [==============================] - 4s 155us/sample - loss: 0.1784 - accuracy: 0.9533 - val_loss: 0.1845 - val_accuracy: 0.9496
Epoch 7/100
27779/27779 [====================